In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor

import cv2

In [2]:
# List of file names
file_names = [
    "Datasets_motor_A/Triangle10Hz-MotorA.csv",
    "Datasets_motor_A/SquareWave-MotorA.csv",
    "Datasets_motor_A/Triangle0.1Hz-MotorA.csv",
    "Datasets_motor_A/Triangle1Hz-MotorA.csv",
    "Datasets_motor_A/Sine10Hz-MotorA.csv",
    "Datasets_motor_A/Sine1Hz-MotorA.csv",
    "Datasets_motor_A/Sine0.1Hz-MotorA.csv",
    "Datasets_motor_A/PRBS9-MotorA.csv",
    "Datasets_motor_A/PRBS7-MotorA.csv",
    "Datasets_motor_A/StepNoLoad-MotorA.csv"
]


In [3]:
# Read and concatenate all datasets
data_frames = []
for file in file_names:
    df = pd.read_csv(file)
    data_frames.append(df)

# Concatenate all dataframes
concatenated_df = pd.concat(data_frames, ignore_index=True)

In [4]:
concatenated_df

,time,encoderCount,Velocity,rawCurrent,Current,rawVoltageA1,rawVoltageB1,VoltageA1,VoltageB1,MotorVoltage,...,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Step_Percentage
0,22620,0,0,510,-0.01,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,31472,0,0,511,-0.03,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,41480,0,0,511,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,51480,0,0,511,0.03,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,61484,0,0,510,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133781,2462480,-590164,497,524,0.89,2,830,0.03,11.96,11.93,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0
133782,2472456,-592692,495,524,0.91,2,831,0.03,11.97,11.94,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0
133783,2482468,-595229,497,521,1.08,3,836,0.04,12.04,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0
133784,2492468,-597764,497,522,0.97,0,835,0,12.03,12.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0


In [5]:
# Drop columns that start with 'Unnamed'
concatenated_df = concatenated_df.loc[:, ~concatenated_df.columns.str.startswith('Unnamed')]

In [6]:
concatenated_df

,time,encoderCount,Velocity,rawCurrent,Current,rawVoltageA1,rawVoltageB1,VoltageA1,VoltageB1,MotorVoltage,MotorStatus,PWM,Step_Percentage
0,22620,0,0,510,-0.01,0,0,0,0,0,0,126,NaN
1,31472,0,0,511,-0.03,0,0,0,0,0,0,171,NaN
2,41480,0,0,511,0,0,0,0,0,0,0,221,NaN
3,51480,0,0,511,0.03,0,0,0,0,0,0,237,NaN
4,61484,0,0,510,0,0,0,0,0,0,0,187,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
133781,2462480,-590164,497,524,0.89,2,830,0.03,11.96,11.93,1,255,100.0
133782,2472456,-592692,495,524,0.91,2,831,0.03,11.97,11.94,1,255,100.0
133783,2482468,-595229,497,521,1.08,3,836,0.04,12.04,12,1,255,100.0
133784,2492468,-597764,497,522,0.97,0,835,0,12.03,12.03,1,255,100.0


In [7]:
concatenated_df = concatenated_df.drop(concatenated_df.columns[-1], axis=1)

In [8]:
concatenated_df

,time,encoderCount,Velocity,rawCurrent,Current,rawVoltageA1,rawVoltageB1,VoltageA1,VoltageB1,MotorVoltage,MotorStatus,PWM
0,22620,0,0,510,-0.01,0,0,0,0,0,0,126
1,31472,0,0,511,-0.03,0,0,0,0,0,0,171
2,41480,0,0,511,0,0,0,0,0,0,0,221
3,51480,0,0,511,0.03,0,0,0,0,0,0,237
4,61484,0,0,510,0,0,0,0,0,0,0,187
...,...,...,...,...,...,...,...,...,...,...,...,...
133781,2462480,-590164,497,524,0.89,2,830,0.03,11.96,11.93,1,255
133782,2472456,-592692,495,524,0.91,2,831,0.03,11.97,11.94,1,255
133783,2482468,-595229,497,521,1.08,3,836,0.04,12.04,12,1,255
133784,2492468,-597764,497,522,0.97,0,835,0,12.03,12.03,1,255


In [9]:
# List of file names
file_names_B = [
    "Datasets_motor_B/Triangle10Hz-MotorB.csv",
    "Datasets_motor_B/SquareWave-MotorB.csv",
    "Datasets_motor_B/Triangle0.1Hz-MotorB.csv",
    "Datasets_motor_B/Triangle1Hz-MotorB.csv",
    "Datasets_motor_B/Sine10Hz-MotorB.csv",
    "Datasets_motor_B/Sine1Hz-MotorB.csv",
    "Datasets_motor_B/Sine0.1Hz-MotorB.csv",
    "Datasets_motor_B/PRBS9-MotorB.csv",
    "Datasets_motor_B/PRBS7-MotorB.csv",
    "Datasets_motor_B/StepNoLoad-MotorB.csv"
    
]



In [10]:
# Read and concatenate all datasets
data_frames_B = []
for file in file_names_B:
    df = pd.read_csv(file)
    data_frames_B.append(df)

# Concatenate all dataframes
concatenated_df_B = pd.concat(data_frames_B, ignore_index=True)

In [11]:
concatenated_df_B

,time,encoderCount,Velocity,rawCurrent,Current,rawVoltageA1,rawVoltageB1,VoltageA1,VoltageB1,MotorVoltage,MotorStatus,PWM,Step_Percentage
0,22620,0,0,510,-0.01,0,0,0,0,0,0,126,NaN
1,31472,0,0,511,0.02,0,0,0,0,0,0,171,NaN
2,41476,0,0,511,0,0,0,0,0,0,0,221,NaN
3,51480,0,0,511,0.03,0,0,0,0,0,0,237,NaN
4,61484,0,0,513,-0.01,0,0,0,0,0,0,187,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
145492,2462448,-581794,488,523,0.84,3,831,0.04,11.97,11.93,1,255,100.0
145493,2472456,-584292,489,523,0.84,3,834,0.04,12.01,11.97,1,255,100.0
145494,2482460,-586788,489,526,0.94,0,831,0,11.97,11.97,1,255,100.0
145495,2492492,-589291,490,524,0.9,0,831,0,11.97,11.97,1,255,100.0


In [12]:
# Drop columns that start with 'Unnamed'
concatenated_df_B = concatenated_df_B.loc[:, ~concatenated_df_B.columns.str.startswith('Unnamed')]

In [13]:
concatenated_df_B

,time,encoderCount,Velocity,rawCurrent,Current,rawVoltageA1,rawVoltageB1,VoltageA1,VoltageB1,MotorVoltage,MotorStatus,PWM,Step_Percentage
0,22620,0,0,510,-0.01,0,0,0,0,0,0,126,NaN
1,31472,0,0,511,0.02,0,0,0,0,0,0,171,NaN
2,41476,0,0,511,0,0,0,0,0,0,0,221,NaN
3,51480,0,0,511,0.03,0,0,0,0,0,0,237,NaN
4,61484,0,0,513,-0.01,0,0,0,0,0,0,187,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
145492,2462448,-581794,488,523,0.84,3,831,0.04,11.97,11.93,1,255,100.0
145493,2472456,-584292,489,523,0.84,3,834,0.04,12.01,11.97,1,255,100.0
145494,2482460,-586788,489,526,0.94,0,831,0,11.97,11.97,1,255,100.0
145495,2492492,-589291,490,524,0.9,0,831,0,11.97,11.97,1,255,100.0


In [14]:
concatenated_df_B = concatenated_df_B.drop(concatenated_df_B.columns[-1], axis=1)

In [15]:
concatenated_df_B

,time,encoderCount,Velocity,rawCurrent,Current,rawVoltageA1,rawVoltageB1,VoltageA1,VoltageB1,MotorVoltage,MotorStatus,PWM
0,22620,0,0,510,-0.01,0,0,0,0,0,0,126
1,31472,0,0,511,0.02,0,0,0,0,0,0,171
2,41476,0,0,511,0,0,0,0,0,0,0,221
3,51480,0,0,511,0.03,0,0,0,0,0,0,237
4,61484,0,0,513,-0.01,0,0,0,0,0,0,187
...,...,...,...,...,...,...,...,...,...,...,...,...
145492,2462448,-581794,488,523,0.84,3,831,0.04,11.97,11.93,1,255
145493,2472456,-584292,489,523,0.84,3,834,0.04,12.01,11.97,1,255
145494,2482460,-586788,489,526,0.94,0,831,0,11.97,11.97,1,255
145495,2492492,-589291,490,524,0.9,0,831,0,11.97,11.97,1,255


In [16]:
concatenated_df.isnull().mean() * 100

time            10.959293
encoderCount    10.973495
Velocity        11.087857
rawCurrent      11.095331
Current         11.095331
rawVoltageA1    11.095331
rawVoltageB1    11.095331
VoltageA1       11.095331
VoltageB1       11.095331
MotorVoltage    11.095331
MotorStatus     11.095331
PWM             11.095331
dtype: float64

In [17]:
concatenated_df_B.isnull().mean()* 100

time            7.651017
encoderCount    7.665450
Velocity        7.804972
rawCurrent      7.804972
Current         7.804972
rawVoltageA1    7.804972
rawVoltageB1    7.804972
VoltageA1       7.804972
VoltageB1       7.804972
MotorVoltage    7.804972
MotorStatus     7.804972
PWM             7.804972
dtype: float64

In [66]:
# drop columns that are not need it "Motor B"
drop_columns = ['time', 'encoderCount', 'Velocity', 'rawCurrent', 'Current','MotorStatus']
concatenated_df = concatenated_df.drop(columns=drop_columns, axis=1)

In [67]:
# drop columns that are not need it "Motor B"
drop_columns = ['time', 'encoderCount', 'Velocity', 'rawCurrent', 'Current','MotorStatus']
concatenated_df_B = concatenated_df_B.drop(columns=drop_columns, axis=1)

In [68]:
concatenated_df

,rawVoltageA1,rawVoltageB1,VoltageA1,VoltageB1,MotorVoltage,PWM
0,0,0,0,0,0,126
1,0,0,0,0,0,171
2,0,0,0,0,0,221
3,0,0,0,0,0,237
4,0,0,0,0,0,187
...,...,...,...,...,...,...
277365,NaN,NaN,NaN,NaN,NaN,NaN
277366,NaN,NaN,NaN,NaN,NaN,NaN
277367,NaN,NaN,NaN,NaN,NaN,NaN
277368,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
concatenated_df_B

,rawVoltageA1,rawVoltageB1,VoltageA1,VoltageB1,MotorVoltage,PWM
0,0,0,0,0,0,126
1,0,0,0,0,0,171
2,0,0,0,0,0,221
3,0,0,0,0,0,237
4,0,0,0,0,0,187
...,...,...,...,...,...,...
244612,NaN,NaN,NaN,NaN,NaN,NaN
244613,NaN,NaN,NaN,NaN,NaN,NaN
244614,NaN,NaN,NaN,NaN,NaN,NaN
244615,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
concatenated_df.isnull().mean() * 100

rawVoltageA1    29.595847
rawVoltageB1    29.595847
VoltageA1       29.595847
VoltageB1       29.595847
MotorVoltage    29.595847
PWM             29.595847
dtype: float64

In [71]:
concatenated_df_B.isnull().mean()* 100

rawVoltageA1    17.482023
rawVoltageB1    17.482023
VoltageA1       17.482023
VoltageB1       17.482023
MotorVoltage    17.482023
PWM             17.482023
dtype: float64

In [72]:
concatenated_df_B.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244617 entries, 0 to 244616
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   rawVoltageA1  201853 non-null  object
 1   rawVoltageB1  201853 non-null  object
 2   VoltageA1     201853 non-null  object
 3   VoltageB1     201853 non-null  object
 4   MotorVoltage  201853 non-null  object
 5   PWM           201853 non-null  object
dtypes: object(6)
memory usage: 11.2+ MB


In [18]:
object_columns = concatenated_df.select_dtypes(include=['object']).columns

# Convert columns to float, handling errors
for col in object_columns:
    concatenated_df[col] = pd.to_numeric(concatenated_df[col], errors='coerce')
    concatenated_df_B[col] = pd.to_numeric(concatenated_df_B[col], errors='coerce')

In [19]:
# Convert these columns to float
concatenated_df[object_columns] = concatenated_df[object_columns].astype(float)
concatenated_df_B[object_columns] = concatenated_df_B[object_columns].astype(float)

In [20]:
concatenated_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133786 entries, 0 to 133785
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   time          118762 non-null  float64
 1   encoderCount  118870 non-null  float64
 2   Velocity      118762 non-null  float64
 3   rawCurrent    118761 non-null  float64
 4   Current       118761 non-null  float64
 5   rawVoltageA1  118761 non-null  float64
 6   rawVoltageB1  118761 non-null  float64
 7   VoltageA1     118761 non-null  float64
 8   VoltageB1     118761 non-null  float64
 9   MotorVoltage  118761 non-null  float64
 10  MotorStatus   118761 non-null  float64
 11  PWM           118761 non-null  float64
dtypes: float64(12)
memory usage: 12.2 MB


In [21]:
concatenated_df_B.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145497 entries, 0 to 145496
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   time          133917 non-null  float64
 1   encoderCount  134055 non-null  float64
 2   Velocity      133917 non-null  float64
 3   rawCurrent    133917 non-null  float64
 4   Current       133917 non-null  float64
 5   rawVoltageA1  133917 non-null  float64
 6   rawVoltageB1  133917 non-null  float64
 7   VoltageA1     133917 non-null  float64
 8   VoltageB1     133917 non-null  float64
 9   MotorVoltage  133917 non-null  float64
 10  MotorStatus   133917 non-null  float64
 11  PWM           133917 non-null  float64
dtypes: float64(12)
memory usage: 13.3 MB


In [23]:
concatenated_df.to_csv('Datasets_motor_A/All_MototA_type.csv', index=False)

In [25]:
concatenated_df_B.to_csv('Datasets_motor_B/All_MototB_type.csv', index=False)